In [1]:
workdir = "/SNS/users/lj7/imaging/testmtex"
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/lj7/imaging/testmtex


In [13]:
import os, numpy as np

# Modules

In [2]:
%%file startup.m
addpath('/HFIR/CG1D/shared/mtex/mtex-4.5.2/')
startup_mtex

Overwriting startup.m


In [44]:
%%file odf2VPSC.m
function odf2VPSC(odf,filename,varargin)
% export an ODF to the VPSC format
%
%% Syntax
% export_VPSC(odf,'file.txt','points',points)
%
%% Input
%  odf      - ODF to be exported
%  filename - name of the ascii file
%
%% Options
%  points    - number of orientations
%
%% See also
% ODFImportExport
  
% get number of points
points = get_option(varargin,'points',5000);

% simulate orientations
orientations = calcOrientations(odf,points,varargin{:});

% export
export_VPSC(orientations,filename,varargin{:});


Overwriting odf2VPSC.m


In [16]:
%%file polfig2VPSC.m
function polfig2VPSC(polfigpath, hkls, outpath)
%% Import Script for PoleFigure Data

%% Specify Crystal and Specimen Symmetries
% crystal symmetry
CS = crystalSymmetry('1', [1 1 1], [90,90,90]*degree, 'X||a', 'Y||b*', 'Z||c*', 'color', 'light blue');

% specimen symmetry
SS = specimenSymmetry('1');

% plotting convention
setMTEXpref('xAxisDirection','north');
setMTEXpref('zAxisDirection','outOfPlane');

%% Specify File Names

% path to files
fname = [polfigpath];

%% Specify Miller Indice
nhkl = size(hkls, 1);
h = arrayfun( @(IDX) Miller(mat2cell(hkls(IDX,:), 1, ones(1, 3)), CS), (1:nhkl).', 'uniform', 0);

%% Import the Data

% create a Pole Figure variable containing the data
pf = loadPoleFigure(fname,h,CS,SS,'interface','siemens');

odf = calcODF(pf);
odf2VPSC(odf, outpath);

Writing polfig2VPSC2.m


# Start matlab

In [7]:
import matlab.engine, matlab
_ml_engine = matlab.engine.start_matlab()

In [23]:
hkls = matlab.double([[1,1,1], [2,0,0], [2,2,0], [2,2,2] ])

In [32]:
import StringIO
out = StringIO.StringIO()
err = StringIO.StringIO()

_ml_engine.polfig2VPSC(
    "/SNS/users/lj7/dv/imaging/braggedgemodeling/tests/data/VULCAN.rpf", 
    hkls,
    "tmp.txt",
    nargout=0, stdout=out, stderr=err
)

In [35]:
print out.getvalue()

------ MTEX -- PDF to ODF inversion ------------------
Call c-routine
initialize solver
start iteration
error: 8.9615E-02 2.9824E-02 2.3061E-02 2.0660E-02 1.9165E-02 1.8185E-02 1.7371E-02 1.6778E-02 1.6254E-02 1.5831E-02 1.5434E-02 
Finished PDF-ODF inversion.
error: 1.5434E-02
alpha: 1.0301E+00 9.2333E-01 1.0223E+00 1.0214E+00 
required time: 5s
ghost correction
calculate with fixed background 0.49
initialize solver
start iteration
error: 2.7973E-01 1.0409E-01 8.0557E-02 7.3001E-02 6.7023E-02 6.3385E-02 6.0057E-02 5.7743E-02 5.5601E-02 5.4001E-02 5.2487E-02 
Finished PDF-ODF inversion.
error: 5.2487E-02
alpha: 1.0334E+00 9.2192E-01 1.0099E+00 1.0255E+00 



In [36]:
print err.getvalue()

# Exit matlab

In [6]:
_ml_engine.exit()